#**Autoencoder Model for Word Embedding**

#I. Import necessary things

In [111]:
import tensorflow as tf
from tensorflow import keras
import re
import pandas as pd
import nltk
import numpy as np
from sklearn import model_selection, preprocessing
from scipy.spatial import distance

nltk.download('stopwords')
nltk.download('punkt')


from google.colab import drive

drive.mount('/content/gdrive')

PATH = '/content/gdrive/My Drive/1970513_Assignment3/'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [112]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1170044513571322232
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 563790852000977044
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11705275168174313101
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15695549568
locality {
  bus_id: 1
  links {
  }
}
incarnation: 718217788856693086
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [113]:
df = pd.read_csv(PATH + 'bbc_news_train.csv')
df = df.head(200)
# split the dataset into training and test datasetss
data_x = df['Text'].values
data_y = df['Category'].values
data_x

array(['worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom.

In [114]:
print(data_x.shape)
print(data_x[0])

(200,)
worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. 

#II. Load corpus and preprocess it

In [115]:
def text_cleaner(text):
    # lower case text
    text = text.lower()
    text = re.sub(r"'s\b","",text)
    # remove punctuations
    text = re.sub("[^a-zA-Z]", " ", text)
    return text

In [116]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

def preprocess_text(corpus):
    all_tokens = []
    for doc in corpus:
        clean_doc = text_cleaner(doc)
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(clean_doc)
        all_tokens += word_tokens
    return [w for w in all_tokens if not w in stop_words]

corpus_token =  preprocess_text(data_x)
print(corpus_token)
print(len(corpus_token))

['worldcom', 'ex', 'boss', 'launches', 'defence', 'lawyers', 'defending', 'former', 'worldcom', 'chief', 'bernie', 'ebbers', 'battery', 'fraud', 'charges', 'called', 'company', 'whistleblower', 'first', 'witness', 'cynthia', 'cooper', 'worldcom', 'ex', 'head', 'internal', 'accounting', 'alerted', 'directors', 'irregular', 'accounting', 'practices', 'us', 'telecoms', 'giant', 'warnings', 'led', 'collapse', 'firm', 'following', 'discovery', 'bn', 'bn', 'accounting', 'fraud', 'mr', 'ebbers', 'pleaded', 'guilty', 'charges', 'fraud', 'conspiracy', 'prosecution', 'lawyers', 'argued', 'mr', 'ebbers', 'orchestrated', 'series', 'accounting', 'tricks', 'worldcom', 'ordering', 'employees', 'hide', 'expenses', 'inflate', 'revenues', 'meet', 'wall', 'street', 'earnings', 'estimates', 'ms', 'cooper', 'runs', 'consulting', 'business', 'told', 'jury', 'new', 'york', 'wednesday', 'external', 'auditors', 'arthur', 'andersen', 'approved', 'worldcom', 'accounting', 'early', 'said', 'andersen', 'given', 'g

In [117]:
dictionary = sorted(list(set(corpus_token)))
mapping = dict((c, i) for i, c in enumerate(dictionary))
print(dictionary)
print(len(dictionary))

['aaa', 'aaron', 'abacus', 'abandoned', 'abandoning', 'abbas', 'abbott', 'abbreviated', 'abc', 'abi', 'abilities', 'ability', 'able', 'abolish', 'abortionist', 'abroad', 'absent', 'absolute', 'absolutely', 'absorb', 'absorbed', 'abstain', 'abu', 'abundantly', 'abuse', 'ac', 'academy', 'accelerate', 'accelerated', 'accept', 'acceptable', 'accepted', 'accepting', 'accepts', 'access', 'accessed', 'accessible', 'accessing', 'accidental', 'acclaimed', 'accolades', 'accompanied', 'according', 'account', 'accountability', 'accountable', 'accountancy', 'accounting', 'accounts', 'accurate', 'accusations', 'accused', 'accusing', 'achievable', 'achieve', 'achieved', 'achievement', 'achievements', 'achieving', 'achilles', 'achtung', 'acknowledge', 'acknowledged', 'acknowledges', 'acquainted', 'acquisitions', 'acre', 'acrimony', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'active', 'activists', 'activities', 'activity', 'actor', 'actors', 'actress', 'acts', 'actual', 'actually', 'ad', 

##Exploratory

In [118]:
from collections import Counter

word_counts = Counter(corpus_token)
print(word_counts.most_common(25))


[('said', 612), ('mr', 239), ('would', 208), ('year', 204), ('new', 194), ('also', 178), ('people', 163), ('last', 146), ('us', 141), ('could', 135), ('world', 129), ('one', 129), ('first', 127), ('two', 123), ('time', 121), ('uk', 105), ('years', 93), ('best', 88), ('three', 81), ('bn', 77), ('well', 77), ('government', 74), ('make', 74), ('good', 73), ('high', 71)]


#III. Convert corpus to one-hot vectors & Define embedding dim

In [119]:
embedding_dim = 300
vocab_size = len(dictionary)
print(vocab_size)

8732


In [120]:
corpus_encode = [mapping[x] for x in corpus_token]
onehot_corpus = keras.utils.to_categorical(corpus_encode, num_classes=vocab_size)
print(onehot_corpus)
print(onehot_corpus.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(41816, 8732)


#IV. Define Autoencoder model

In [121]:
ae_model = keras.Sequential()
ae_model.add(keras.Input(shape=(vocab_size,)))
ae_model.add(keras.layers.Dense(embedding_dim, activation='relu'))
ae_model.add(keras.layers.Dense(vocab_size, activation='softmax'))

ae_model.compile(optimizer='adam', loss='categorical_crossentropy')

ae_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 300)               2619900   
_________________________________________________________________
dense_11 (Dense)             (None, 8732)              2628332   
Total params: 5,248,232
Trainable params: 5,248,232
Non-trainable params: 0
_________________________________________________________________


#V. Train AE model

In [122]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
ae_model.fit(x=onehot_corpus, y=onehot_corpus, batch_size=128, epochs=200, callbacks=[early_stopping])
ae_model.save(PATH + "ae_result/ae_model.h5")

Epoch 1/200
327/327 [==============================] - 2s 7ms/step - loss: 7.9009
Epoch 2/200
327/327 [==============================] - 2s 7ms/step - loss: 5.2596
Epoch 3/200
327/327 [==============================] - 2s 7ms/step - loss: 3.1834
Epoch 4/200
327/327 [==============================] - 2s 7ms/step - loss: 2.2945
Epoch 5/200
327/327 [==============================] - 2s 7ms/step - loss: 1.8161
Epoch 6/200
327/327 [==============================] - 2s 7ms/step - loss: 1.4956
Epoch 7/200
327/327 [==============================] - 2s 7ms/step - loss: 1.2544
Epoch 8/200
327/327 [==============================] - 2s 7ms/step - loss: 1.0602
Epoch 9/200
327/327 [==============================] - 2s 7ms/step - loss: 0.9125
Epoch 10/200
327/327 [==============================] - 2s 7ms/step - loss: 0.7829
Epoch 11/200
327/327 [==============================] - 2s 7ms/step - loss: 0.6770
Epoch 12/200
327/327 [==============================] - 2s 7ms/step - loss: 0.5867
Epoch 13/200


#=======================================================================

# **How to use trained model**

#=======================================================================

#VI. Load trained model & Get output of 1st FC layer

In [123]:
reconstructed_model = keras.models.load_model(PATH + "ae_result/ae_model.h5")
we_model = keras.models.Model(inputs=reconstructed_model.inputs, outputs=reconstructed_model.get_layer(index=0).output)

Define function to encode one-hot list of words

In [124]:
def encode_onehot(mapping, list_words):
  output = []

  for word in list_words:
    word_vector = [0 for _ in range(vocab_size)]

    if word in mapping.keys():
      word_index = mapping[word]
      word_vector[word_index] = 1
    
    output.append(word_vector)
  
  return output

#VII. Test model

In [125]:
input_sentence = ['fed chief warning on us deficit federal reserve chairman alan greenspan has warned that allowing huge us budget deficits to continue could have severe consequences']
preprocess_sentence = preprocess_text(input_sentence)
onehot_sentence = encode_onehot(mapping, preprocess_sentence)

embedded_sentence = we_model.predict(onehot_sentence)
print(embedded_sentence)
print(embedded_sentence.shape)

[[6.2592053 6.740576  6.691345  ... 5.112497  6.381433  6.0511727]
 [6.0440283 6.7117224 6.691658  ... 5.5404396 6.358134  5.8944745]
 [5.6388803 6.7201242 6.0609436 ... 4.8970704 5.791489  6.0965996]
 ...
 [6.400837  6.869797  6.7731605 ... 4.75828   5.605503  6.7950616]
 [5.603598  6.656809  6.6256604 ... 5.4905877 6.3058157 6.4057965]
 [5.5881367 6.641713  6.4635963 ... 5.387885  5.7824836 5.874913 ]]
(20, 300)


In [132]:
interesting_words, counter = zip(*word_counts.most_common(10))
print(interesting_words)

preprocess_words = preprocess_text(' '.join(interesting_words))
onehot_words = encode_onehot(mapping, preprocess_words)

words_eb = we_model.predict(onehot_words)

print("OUTPUT EMBEDDING")

for i in range(len(words_eb)):
  i_next = (i+1)%len(words_eb)
  print(words_eb[i])

print("OUTPUT DISTANCE")
for i in range(len(words_eb)):
  i_next = (i+1)%len(words_eb)
  print(str(i) + " vs " + str(i_next)  + ": " + str(distance.euclidean(words_eb[i], words_eb[i_next])))

print("OUTPUT AVG DISTANCE WITH OTHERS")
for i in range(len(words_eb)):
  words_eb_ = list(words_eb.copy())
  words_eb_.pop(i)
  dist_sum = 0
  for k in range(len(words_eb_)):
    dist_sum += distance.euclidean(words_eb[i], words_eb_[k])
  dist_avg = dist_sum/len(words_eb)
  print("The avg distance of " + str(i) + ": " + str(dist_avg))


('said', 'mr', 'would', 'year', 'new', 'also', 'people', 'last', 'us', 'could')
OUTPUT EMBEDDING
[5.5818934 6.1073747 6.6952715 5.9856725 6.2167034 6.2077093 5.841765
 5.819428  5.588465  6.321094  6.0090437 5.938832  8.170162  6.81075
 6.4510016 5.7494497 5.8391733 6.094695  6.85739   6.8377447 7.2049594
 5.7137823 7.185772  6.5054417 6.3255825 6.076917  6.450775  6.0203605
 5.8792744 6.031188  5.801085  7.1464252 5.8551126 6.2140565 6.404151
 6.2808967 5.806934  7.8335032 6.316636  6.209599  5.996045  5.9311914
 6.4855523 7.2989006 6.680565  6.69344   5.9079113 6.266181  5.7992764
 6.3382044 6.703617  5.988301  6.9352627 7.1196127 6.6504097 5.3860116
 5.9023623 6.3890324 6.532682  6.599966  5.875207  6.1152687 7.61481
 6.2711554 5.8140526 6.41072   6.3817863 5.9863825 6.091671  6.653092
 5.9088964 6.1280427 5.856859  5.837841  7.627105  5.603671  5.861885
 5.9004397 6.5133667 5.931252  5.972762  6.0649858 6.4866085 6.850458
 6.6151276 5.644233  6.3069    6.603452  6.054292  6.5443344

- Khoảng cách giữa các từ khác nhau tương tự nhau, không thể hiện rõ sự khác biệt
- Khoảng cách trung bình giữa của 1 từ so với các từ khác cũng không có sự khác nhau rõ rệt
- Những từ khác nhau cho ra vector khác nhau